In [301]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")

In [587]:
#Load dataset
CPS=pd.read_csv('data/CPSData.csv')

In [466]:
#Among the interviewees with a value reported for the industry variable, what
#is the most common industry of employment?
new=CPS[CPS['Industry'].notnull()]
new.groupby('Industry')['Age'].count().sort_values(ascending=False).index[0]

'Educational and health services'

In [471]:
#Which state has the fewest interviewees?
print(CPS.groupby('State')['Age'].count().sort_values().index[0])
#Which state has the most interviewees?
print(CPS.groupby('State')['Age'].count().sort_values(ascending=False).index[0])

New Mexico
California


In [485]:
#Which proportion of interviewees are citizens of the United States?
"%.2f" % float(1-len(CPS[CPS['Citizenship']=="Non-Citizen"])/len(CPS))

'0.94'

In [500]:
#Which races are there at least 250 interviewees in the CPS dataset
#of Hispanic ethnicity?
new=CPS[CPS['Hispanic']==1]
new=new.groupby("Race").count()['Age']
new[new>250]

Race
American Indian      304
Black                621
Multiracial          448
White              16731
Name: Age, dtype: int64

In [504]:
#Which variables have at least one interviewee with a missing value?
CPS.isnull().any()

PeopleInHousehold     False
Region                False
State                 False
MetroAreaCode          True
Age                   False
Married                True
Sex                   False
Education              True
Race                  False
Hispanic              False
CountryOfBirthCode    False
Citizenship           False
EmploymentStatus       True
Industry               True
dtype: bool

In [530]:
#The Married variable being missing is not related to which other variable?
print(len(CPS[CPS['Married'].isnull() & CPS['Education'].isnull()]))
print(len(CPS[CPS['Married'].isnull() & CPS['Region'].isnull()]))

25338
0


In [544]:
#How many states had all interviewee living in a non-metropolitan area (missing MetroAreaCode Value)?
missing=CPS[CPS['MetroAreaCode'].isnull()].groupby('State').count()['Age']
total=CPS.groupby('State').count()['Age']
tmp=total-missing
tmp[tmp==0]

State
Alaska     0.0
Wyoming    0.0
Name: Age, dtype: float64

In [553]:
#How many states had all interviewees living in a metropolitan area?
notMissing=CPS[CPS['MetroAreaCode'].notnull()].groupby('State').count()['Age']
total=CPS.groupby('State').count()['Age']
tmp=total-notMissing
tmp[tmp==0]

State
District of Columbia    0.0
New Jersey              0.0
Rhode Island            0.0
Name: Age, dtype: float64

In [562]:
#Which region of the United States has the largest proportion of interviewees
#living in a non-metro area?
notMissing=CPS[CPS['MetroAreaCode'].notnull()].groupby('Region').count()['Age']
total=CPS.groupby('Region').count()['Age']
(notMissing/total).sort_values(ascending=False).index[3]

'Midwest'

In [574]:
#Which state has a proportion of interviewees living in a non-metropolitan area 
#closest to 30%?
CPS.groupby('State')['MetroAreaCode'].apply(lambda x: x.isnull().mean()).sort_values()

State
Rhode Island            0.000000
New Jersey              0.000000
District of Columbia    0.000000
California              0.020484
Florida                 0.039231
Massachusetts           0.064922
Maryland                0.069375
New York                0.080608
Connecticut             0.085684
Illinois                0.112219
Colorado                0.129915
Arizona                 0.131545
Nevada                  0.133082
Texas                   0.143705
Louisiana               0.161379
Pennsylvania            0.174300
Michigan                0.178257
Washington              0.181319
Georgia                 0.198432
Virginia                0.198442
Utah                    0.210098
Oregon                  0.218219
Delaware                0.233966
New Mexico              0.245009
Hawaii                  0.249166
Ohio                    0.251223
Alabama                 0.258721
Indiana                 0.291417
Wisconsin               0.299330
South Carolina          0.313028
Minn

In [575]:
#Load mapping dictionaries
MetroAreaMap=pd.read_csv("data/MetroAreaCodes.csv")
CountryMap=pd.read_csv('data/CountryCodes.csv')

In [588]:
#Merge mapping dictionaries to main dataset
CPS=pd.merge(CPS,MetroAreaMap,left_on="MetroAreaCode",right_on="Code").drop('Code',axis=1)
CPS=pd.merge(CPS,CountryMap,left_on="CountryOfBirthCode",right_on="Code").drop('Code',axis=1)

In [594]:
#Which metro area has the largest number of interviewees?
CPS.groupby('MetroArea').count()['Age'].argmax()

'New York-Northern New Jersey-Long Island, NY-NJ-PA'

In [596]:
#Which metropolitan area has the highest proportion of Hispanic ethnicity?
CPS.groupby('MetroArea')['Hispanic'].mean().argmax()

'Laredo, TX'

In [604]:
#Find the number of metropolitan areas in the United States from which
#at least 20% of interviewees are Asian
asian=CPS.groupby('MetroArea').apply(lambda x: (x.Race=="Asian").mean())
len(asian[asian>0.2])

4

In [610]:
#Which metro area has smallest proportion of interviees who have received no high 
#school dipoma?
new=CPS[CPS.Education.notnull()].groupby('MetroArea').apply(lambda x: (x.Education=="No high school diploma").mean())
new.argmin()

'Iowa City, IA'

In [635]:
#Outside of America, which country was the most common palce of birth
notNA=CPS[CPS.Country.isin(['Canada','United States','Mexico'])==False]
notNA.groupby('Country').count()['Age'].argmax()

'Philippines'

In [644]:
#What proportion of the interviewees from New York-Northern New Jersey-Long Island, NY-NJ-PA
#have a country of birth that is not the United States?
notUS=CPS[(CPS.Country!='United States') & (CPS.MetroArea=='New York-Northern New Jersey-Long Island, NY-NJ-PA')]
local=CPS[CPS['MetroArea']=="New York-Northern New Jersey-Long Island, NY-NJ-PA"]
len(notUS)/len(local)

0.30866025166543304

In [650]:
#Which metro area has the largest number of interviewees with a country of birth in India?
print(CPS.groupby("MetroArea").apply(lambda x: (x.Country=='India').mean()).argmax())
#In Brazil?
print(CPS.groupby('MetroArea').apply(lambda x: (x.Country=='Brazil').mean()).argmax())
#In Somalia
print(CPS.groupby('MetroArea').apply(lambda x: (x.Country=='Somalia').mean()).argmax())

Warner Robins, GA
Danbury, CT
St. Cloud, MN
